# Simulate AWS DMS output

In my thesis, the input files are pretended to be written by AWS DMS. Instead of actually creating a relational database and extracting it using AWS DMS, I have chosen to directly create the output files.

In [1]:
import pandas as pd
import random
import os
from helpers.paths import PathMerger
from faker import Faker
from datetime import datetime, timedelta, date, timezone
from dateutil import tz
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

fake = Faker()

## Functions

In [2]:
def generate_customers(first_registration:datetime):
    customers = []
    
    # Memorized value for the creation time of the previous customer.
    created_prev = None
    
    # A single hard-coded customer for reference with an id of 0
    customer = {
        'id': 0, 
        'username': 'janisourander@kamk.fi',
        'password': fake.md5(),
        'created': datetime(1290, 1, 15, 0, 0, 0),
        'modified': datetime(1970, 2, 20, 12, 34, 56),
    }
    
    customers.append(customer)

    for i in range(9):
        
        if created_prev is None:
            # The first ever customer was created at 13:37, 15th of Jan 2020.
            created = first_registration
        else:
            # Each other customer were created n second later than the previous.
            created = created_prev + timedelta(seconds=random.randrange(1, 3600))
        
        customer = {
            'id': len(customers), 
            'username': fake.unique.ascii_email(),
            'password': fake.md5(),
            'created': created,
            'modified': created + timedelta(seconds=random.randrange(0, 3600 * 24 * 365)),
        }
        
        customers.append(customer)
        
        # Set current created datetime as previous
        created_prev = created

    return pd.DataFrame.from_records(customers)
    
    
def generate_device_models():
    devices = [
        {
            'id': 0, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Red',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 1),
            'modified': datetime(2010, 3, 21, 12, 0, 1),
        },
        {
            'id': 1, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Black',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 2),
            'modified': datetime(2010, 3, 21, 12, 0, 2),
        },
        {
            'id': 2, 
            'release_date': date(2010, 11, 1),
            'name': 'Super Gadget 100',
            'color': 'Pink',
            'description': 'lorem ipsum',
            'created': datetime(2010, 8, 5, 7, 0, 0),
            'modified': datetime(2010, 8, 5, 7, 0, 0),
        },
        {
            'id': 3, 
            'release_date': date(2018, 5, 13),
            'name': 'Super Gadget 200',
            'color': 'White',
            'description': 'lorem ipsum',
            'created': datetime(2018, 3, 20, 12, 1, 1),
            'modified': datetime(2018, 3, 20, 12, 1, 1),
        },
    ]
    
    return pd.DataFrame.from_records(devices)


def generate_devices(customers: pd.DataFrame, devices: pd.DataFrame):

    # List of unique model numbers
    unique_device_models = list(devices.id.unique())
    
    # Container
    devices = []
    i = 0

    for (_, cust) in customers.iterrows():
        
        # Fetch the creation date of user and add some
        created = cust.created + timedelta(seconds=random.randrange(1, 3600))
        
        # Create 1 or 2 devices per user by random
        for x in range(random.randrange(1, 3)):
            device = {
                'id': len(devices), 
                'customer_fk': cust.id,
                'model_fk': random.choice(unique_device_models),
                'serial_number': fake.ssn(),
                'created': created,
                'modified': created + timedelta(seconds=random.randrange(1, 3600)),
            }

            devices.append(device)
    
    return pd.DataFrame.from_records(devices)

        
def generate_customer_details(customers: pd.DataFrame):
    
    # Container
    details = []
    
    for (_, cust) in customers.iterrows():
        detail = {
            'id': len(details),
            'customer_fk': cust.id,
            'birthday': fake.date_of_birth(),
            'language': fake.language_code(),
            'street_address': fake.street_address(),
            'postal_code': fake.postalcode(),
            'city': fake.city(),
            'country': fake.country_code(),
            'phone_number': fake.phone_number(),
            'created': cust.created,
            'modified': cust.created,
        }
        
        details.append(detail)
        
    return pd.DataFrame.from_records(details)

## Create Directories

In [5]:
# Mapping
table_paths = {}

# Map of combinations to create: (db, table)
tables = [
    ('customers', 'customers'), 
    ('customers', 'customer_details'),
    ('devices', 'device_models'),
    ('devices', 'devices')
]

for db, table in tables:
    dt = f'{db}.{table}'
    sp = PathMerger(db, table).staging
    dp = os.path.join(sp, 'LOAD00000001.parquet')
    
    # Create dirs
    if not os.path.exists(sp):
        os.makedirs(sp)
    
    # Append
    table_paths[dt] = dp

## Generate Datasets

### Customers

In [6]:
# Generate
df_c = generate_customers(first_registration=datetime(2020, 1, 15, 13, 37, 0))

# Write
df_c.to_parquet(table_paths['customers.customers'])

# Show
df_c

,id,username,password,created,modified
0,0,janisourander@kamk.fi,ae6743f425fd616c01f4a046c4873635,1290-01-15 00:00:00,1970-02-20 12:34:56
1,1,wrivera@lowe.org,3e074ece5f07a59ac540bdf80719dd8f,2020-01-15 13:37:00,2020-11-29 19:14:52
2,2,jill35@walker-thornton.net,97c19456e44160c1f44729b13ed80a68,2020-01-15 14:11:50,2020-05-23 15:01:53
3,3,henrystephanie@ortega-mahoney.com,7abb90c2f709cdad1e380072668c00ae,2020-01-15 14:52:26,2020-04-02 23:49:23
4,4,zwilliams@hotmail.com,fe31516750fd66761c57b74dc7b2f223,2020-01-15 15:15:46,2020-04-06 01:25:39
5,5,jjohnson@gmail.com,524ca768fdc9f229f9eb456b083a785d,2020-01-15 16:11:26,2020-07-10 20:22:58
6,6,vbass@ortiz.biz,74a8165ea2bf93efcb90e3f59619099b,2020-01-15 16:41:44,2020-04-04 15:36:37
7,7,eric56@flores.com,95b5b8d40d9b6fbef8e7298bb8be1bf0,2020-01-15 17:07:42,2020-04-13 19:31:43
8,8,paulstephens@cortez.biz,16125553478552531a58e3861153255d,2020-01-15 18:01:17,2020-08-07 07:45:25
9,9,jonessarah@carson.org,33dc5f8f22a6cd472aa7628e1ef32601,2020-01-15 18:40:30,2020-07-07 02:09:16


### Device Models

In [7]:
# Generate
df_dm = generate_device_models()

# Write
df_dm.to_parquet(table_paths['devices.device_models'])

# Show
df_dm

,id,release_date,name,color,description,created,modified
0,0,2010-05-15,Super Gadget 100,Red,lorem ipsum,2010-03-21 12:00:01,2010-03-21 12:00:01
1,1,2010-05-15,Super Gadget 100,Black,lorem ipsum,2010-03-21 12:00:02,2010-03-21 12:00:02
2,2,2010-11-01,Super Gadget 100,Pink,lorem ipsum,2010-08-05 07:00:00,2010-08-05 07:00:00
3,3,2018-05-13,Super Gadget 200,White,lorem ipsum,2018-03-20 12:01:01,2018-03-20 12:01:01


### Devices

In [8]:
# Generate devices based on customers and device models
df_d = generate_devices(df_c, df_dm)

# Write
df_dm.to_parquet(table_paths['devices.devices'])

# Show
df_d

,id,customer_fk,model_fk,serial_number,created,modified
0,0,0,3,358-71-8703,1290-01-15 00:13:32,1290-01-15 00:45:35
1,1,1,2,329-01-7452,2020-01-15 14:13:47,2020-01-15 14:54:29
2,2,2,0,227-77-6653,2020-01-15 14:27:01,2020-01-15 15:04:17
3,3,2,3,079-38-6956,2020-01-15 14:27:01,2020-01-15 14:47:03
4,4,3,0,699-64-3086,2020-01-15 15:43:34,2020-01-15 16:31:52
5,5,3,3,389-49-9299,2020-01-15 15:43:34,2020-01-15 16:04:36
6,6,4,1,021-64-6984,2020-01-15 15:51:18,2020-01-15 16:12:50
7,7,4,1,142-60-0264,2020-01-15 15:51:18,2020-01-15 16:28:03
8,8,5,0,215-60-0474,2020-01-15 16:40:05,2020-01-15 16:43:11
9,9,6,0,272-06-5862,2020-01-15 17:03:20,2020-01-15 17:37:36


### Customer_details

In [9]:
# Generate customer details. One per customer.
df_cd = generate_customer_details(df_c)

# Write
df_dm.to_parquet(table_paths['customers.customer_details'])

df_cd

,id,customer_fk,birthday,language,street_address,postal_code,city,country,phone_number,created,modified
0,0,0,1984-10-07,cmn,9194 Peter Radial Suite 123,44227,North Joseph,ME,001-232-513-4903x429,1290-01-15 00:00:00,1290-01-15 00:00:00
1,1,1,2010-07-18,fy,12498 Smith Walk,40787,Scottton,GT,790.397.3502,2020-01-15 13:37:00,2020-01-15 13:37:00
2,2,2,1946-11-23,ms,04108 Hawkins Road Suite 393,78663,New Christine,TR,334.527.3075x98743,2020-01-15 14:11:50,2020-01-15 14:11:50
3,3,3,2019-11-18,de,9823 Patel Square,89653,Lake Nicole,CZ,368-395-0368x2477,2020-01-15 14:52:26,2020-01-15 14:52:26
4,4,4,1990-04-20,tt,96922 Jenkins Islands,33476,Jonesmouth,TV,(170)491-1077x2869,2020-01-15 15:15:46,2020-01-15 15:15:46
5,5,5,1981-08-10,ml,61306 Kim Fort Suite 670,68181,East Robertland,DM,(278)871-9792,2020-01-15 16:11:26,2020-01-15 16:11:26
6,6,6,1948-07-13,mr,276 Jeff Mall,34795,Santiagotown,NZ,398-830-6361,2020-01-15 16:41:44,2020-01-15 16:41:44
7,7,7,1943-07-20,mg,158 Kelly Island Suite 634,18377,Kimberlymouth,KI,(195)287-4688x39137,2020-01-15 17:07:42,2020-01-15 17:07:42
8,8,8,2017-07-29,gv,2268 Paul Turnpike Suite 635,81625,Patrickside,MN,984-605-3911x9645,2020-01-15 18:01:17,2020-01-15 18:01:17
9,9,9,2007-03-02,mr,7607 Anderson Villages,91052,Rodriguezview,HT,9025366726,2020-01-15 18:40:30,2020-01-15 18:40:30


## List resulting files

In [10]:
# Maximum lenght of a path
max_w = max([len(x[1]) for x in table_paths.items()])

# UNIX epoch start
epoch = datetime(1970, 1, 1, tzinfo=timezone.utc)

for k, f in table_paths.items():

    s = os.stat(f)
    f_size = s.st_size // 1024 
    f_mod = (
        (epoch + timedelta(seconds=s.st_mtime))
        .astimezone(tz.gettz('Europe/Helsinki'))
        .strftime('%y-%m-%d %H:%M:%S')
    )
    
    print(f"{f_mod} | {f:<{max_w}} | {f_size:>2d} KB")

21-08-04 09:03:53 | S3\staging\dms\company_rds\customers\customers\LOAD00000001.parquet        |  5 KB
21-08-04 09:03:58 | S3\staging\dms\company_rds\customers\customer_details\LOAD00000001.parquet |  5 KB
21-08-04 09:03:55 | S3\staging\dms\company_rds\devices\device_models\LOAD00000001.parquet      |  5 KB
21-08-04 09:03:56 | S3\staging\dms\company_rds\devices\devices\LOAD00000001.parquet            |  5 KB
